In [ ]:
import pandas as pd
from core.sql_utils import *
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt

from transform.raw_results.renault_results import *
from transform.insights_results.trendlines import *


## Data

In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT *  FROM  vehicle v	
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join vehicle_data vd 
            on vd.vehicle_id  = v.id
            join oem o 
            on o.id = vm.oem_id
            join battery b 
            on b.id = vm.battery_id
            where o.oem_name in ('renault');"""), con)



dbeaver_df.head()
dbeaver_df['timestamp'] = pd.to_datetime(dbeaver_df['timestamp'])
dbeaver_df.sort_values("timestamp", inplace=True)
dbeaver_df.shape

In [ ]:
px.scatter(dbeaver_df,  x='odometer', y='soh', color="vin")

## Explo soh

In [ ]:
## raw_result renault

raw_results = get_results()

In [ ]:
px.scatter(raw_results,  x='odometer', y='soh', color="vin")

In [ ]:
processed_tss  = ProcessedTimeSeries("renault", filters=[("in_charge", "==", True)] )

In [ ]:
px.scatter(raw_results[raw_results["soc"] > .5]
          , x='odometer', y='soh', color='vin',
           hover_data={"charging_rate": True,
                       "battery_energy": True,
                       "odometer": True})

In [ ]:
raw_results[['soh', 'battery_energy', 'soc']].corr()

### recompute SoH

In [ ]:
from core.sql_utils import *
from core.stats_utils import *
from core.pandas_utils import *
from transform.processed_results.config import *

In [ ]:
def get_processed_results(brand:str) -> DF:
    results =  (
        raw_results_with_discharge
        # Some raw estimations may have inf values, this will make mask_out_outliers_by_interquartile_range and force_monotonic_decrease fail
        # So we replace them by NaNs.
        .assign(soh=lambda df: df["soh"].replace([np.inf, -np.inf], np.nan))
        .sort_values(["vin", "date"])
        .pipe(make_charge_levels_presentable)
        .eval(SOH_FILTER_EVAL_STRINGS[brand])
        .pipe(agg_results_by_update_frequency)
        .groupby('vin', observed=True)
        .apply(make_soh_presentable_per_vehicle, include_groups=False)
        .reset_index(level=0)
        #.pipe(filter_results_by_lines_bounds, VALID_SOH_POINTS_LINE_BOUNDS, logger=logger)
        .sort_values(["vin", "date"])
    )
    results["soh"] = results.groupby("vin", observed=True)["soh"].ffill()
    results["soh"] = results.groupby("vin", observed=True)["soh"].bfill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].ffill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].bfill()
    return results

In [ ]:
processed_results = get_processed_results("renault")
px.scatter(processed_results, x='odometer', y='soh', color='vin', hover_data={})

In [ ]:
px.scatter(dbeaver_df,  x='odometer', y='soh', color="vin")

In [ ]:
px.scatter(dbeaver_df.groupby('vin')[['soh', 'odometer']].mean(), x='odometer', y='soh')

## trendline

In [ ]:
test = dbeaver_df.groupby('vin')[['soh', 'odometer']].mean().dropna()

In [ ]:
def f(x, s, a, b):
    return s + np.log1p(x/b) * a

In [ ]:
# start n'importe ou
coef_, _ = curve_fit(f, test.sort_values('odometer')['odometer'], test.sort_values('odometer')['soh'], maxfev=10000)
y_fit_mean = f(test.sort_values('odometer')['odometer'].values, *coef_)

# passage en (0, 1)
coef_, _ = curve_fit(log_function, pd.concat((test, pd.DataFrame({"soh":[1], 'odometer':[0]}))).sort_values('odometer')['odometer'], pd.concat((test, pd.DataFrame({"soh":[1], 'odometer':[0]}))).sort_values('odometer')['soh'], maxfev=10000)
y_fit = log_function(pd.concat((test, pd.DataFrame({"soh":[1], 'odometer':[0]}))).sort_values('odometer')['odometer'].values, *coef_)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test.sort_values('odometer')['odometer'], y=y_fit , name='Avec contrainte (0,100)'))
fig.add_trace(go.Scatter(x=test.sort_values('odometer')['odometer'], y=y_fit_mean , name='Sans contrainte'))
fig.add_trace(go.Scatter(x=test.sort_values('odometer')['odometer'], y=test.sort_values('odometer')['soh'],  mode='markers',
))

## Trendlines


In [ ]:
trendline, trendline_max, trendline_min = get_trendlines(df_plot)

In [ ]:
def trendline_apply(x, f):
    return eval(f)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=dbeaver_df['odometer'],
    y=dbeaver_df['soh'],
    mode='markers',
    marker_color='rgba(50, 182, 193, .9)',
    name='SoH compute'

    ))

fig.add_trace(go.Scatter(x=dbeaver_df['odometer'].sort_values(), 
                         y=log_function(dbeaver_df['odometer'].sort_values(), *coef_), 
                         line=dict(color='green'), 
                         name='Sans contrainte'))
              
fig.add_trace(go.Scatter(
    x=pd.concat((pd.DataFrame(data={'odometer':[0]}), dbeaver_df['odometer'].sort_values())).values.reshape(501,),
    y=pd.concat((pd.DataFrame(data={'odometer':[1]}), trendline_apply(dbeaver_df['odometer'].sort_values(), trendline['trendline']))).values.reshape(501,),
    mode='lines',
    line=dict(color='red'),
    name='Avec contrainte (0,100)'
))


# fig.add_trace(go.Scatter(
#     x=dbeaver_df['odometer'].sort_values(),
#     y=trendline_apply(dbeaver_df['odometer'].sort_values(), trendline_max['trendline']),
#     mode='lines',
#     line=dict(color='green'),
#     name='upper'
# ))


# fig.add_trace(go.Scatter(
#     x=dbeaver_df['odometer'].sort_values(),
#     y=trendline_apply(dbeaver_df['odometer'].sort_values(), trendline_min['trendline']),
#     mode='lines',
#     line=dict(color='green'),
#     name='lower'
# ))


fig.update_layout(
    width=1000,
    height=600,
    xaxis_title='x',
    yaxis_title='y',
    legend_title='Légende',
    template='plotly_white'
)

fig.show()
